# Solución hidrostática de una estrella formada por un gas ideal de $n$, $p$ y $e$ degenerados

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import CubicSpline, interp1d

## Ecuacion de estado del gas

### Expresiones

Similar al gas degenerado de neutrones, este gas ideal de $n$, $p$ y $e$ degenerados corresponde a la suma de las cantidades obtenidas para cada especie, dado que no interactúan. Así, utilizando los parámetro $x_i = \frac{p_F^i}{m_i c}$, se tiene para la presión, densidad de energía y densidad de número de partículas:

$$ \hat P = \frac{1}{P_0} (A_e f(x_e) + A_p f(x_p) + A_n f(x_n)), $$
$$ \hat \rho = \frac{3}{\rho_0} (A_e g(x_e) + A_p g(x_p) + A_n g(x_n))$$
$$ n = \frac{B_e}{m_e} x_e^3 + \frac{B_p}{m_p} x_p^3 + \frac{B_n}{m_n} x_n^3 \Rightarrow \rho_m = B_e x_e^3 + B_p x_p^3 + B_n x_n^3$$

donde

$$ f(x) = x\sqrt{x^2+1}(2x^2-3) + 3 ln(x+\sqrt{x^2+1}), $$
$$ g(x) = x\sqrt{x^2+1}(2x^2+1) + ln(x+\sqrt{x^2+1}),$$

con

$$ A_i = \frac{\pi m_i^4 c^5}{3h^3},$$
$$ B_i = \frac{8 \pi m_i^4 c^3}{3h^3} = \frac{8}{c^2}A_i.$$

Para resolver este sistema, se asume neutralidad global de la estrella, es decir, empleando la relación $n_i = \frac{8\pi}{3 h^3} (p_F^i)^3$, se tiene:

$$ n_e = n_p \Rightarrow m_e x_e = m_p x_p,$$

así como equilibrio frente al decaimiento $\beta$ inverso, considerando que el potencial químico en el caso completamente degenerado es la energía de fermi $\mu_i=E_F^i = m_i c^2 \left[ 1+ \left(\frac{p_F^i}{mc}\right)^2 \right]^{1/2}$:

$$ \mu_e + \mu_p = \mu_n \Rightarrow m_e(1+x_e^2)^{1/2} + m_p(1+x_p^2)^{1/2} = m_n(1+x_n^2)^{1/2}.$$

De este modo se tienen las siguientes relaciones que permiten resolver con el parametro $x_e$:

$$ x_p = \frac{m_e}{m_p} x_e,$$
$$ x_n = \frac{1}{m_n} \left[ \left( m_e(1+x_e^2)^{1/2} + \left(m_p^2+m_e^2 x_e^2\right)^{1/2} \right)^2 -m_n^2 \right]^{1/2}.$$

### Interpolación de las ecuaciones paramétricas